In [209]:
import json
from functools import partial
from inspect import getsource
from math import log
from sys import argv
from bisect import bisect_left
import re

def checker(nm,ori):
    flag = False
    for key in ori.keys():
        if ori[key] != nm[key]:
            flag = True
            break
    print('Match!!' if flag == False else 'Error!!')

def test(origin_mapping,idx):
    print(len(idx))
    mark = consecu_int_run_length(idx)
    # check part
    _m = mark
    nm = {}
    for i in range(len(keys)):
        nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        for j in range(_m[i+2]):
            nm[keys[_m[i+1]+j]] = values[_m[i]+j]
    checker(nm,origin_mapping)

    sporadic_seg = sporadic_seg_run_length(mark)
    # check part
    _m = sporadic_seg
    nm = {}
    vv = set(values)
    for i in range(len(keys)):
        if keys[i] in values:
            nm[keys[i]] = keys[i]
        #nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        if _m[i+2] != 1:
            for j in range(_m[i+2]):
                nm[keys[_m[i+1]+j]] = values[_m[i]+j]
        else:
            length = len(_m[i+1])
            for j in range(length):
                nm[keys[_m[i+1][j]]] = values[_m[i]+j]
    checker(nm,origin_mapping)

def consecu_int_run_length(idx):
    mark = []
    flag = False
    temp = []
    for i in range(len(k)):
        if i != idx[i]:      
            if flag == False:
                temp = [ i, idx[i], 1 ]
                flag = True
            elif flag == True and idx[i]-(temp[1]+temp[2]) == 0:
                temp[2] += 1
            elif flag == True:
                mark += temp
                temp = [ i, idx[i], 1 ]
                flag = True
        else:
            if flag == True:
                mark += temp
                temp = []
                flag = False
    if len(temp) != 0:
        mark += temp
    return mark

def sporadic_seg_run_length(mark):
    sporadic_seg = [ mark[i:i+3] for i in range(0,len(mark),3)]
    flag = False
    temp = []
    res = []
    for i in range(len(sporadic_seg)):
        if sporadic_seg[i][2] != 1:
            if flag == True:
                res += temp 
                # initialize temp to an empty list, to avoid if temp have something but index is at the end
                temp = []
                flag = False
            res += sporadic_seg[i]
        else:
            # first one
            if flag == False:
                # start index, index list, len
                temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
                flag = True
            else:
                if sporadic_seg[i][0] - sporadic_seg[i-1][0] == 1:
                    temp[1].append(sporadic_seg[i][1]) 
                else:
                    res += temp  
                    temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
    if len(temp) != 0:
        res += temp  
    return res

def classify(origin_mapping):
    same, prefix, infix, postfix, others = [], [], [], [], []
    for k,v in origin_mapping.items():
        index = v.find(k)
        if k == v:
            same += [k]
        elif index != -1:
            if index == 0:
                prefix += [k]
            elif v[index:] == k:
                postfix += [k]
            else:
                infix += [k]
        else:
            others += [k]
    print('same: {}\nprefix: {}\ninfix: {}\npostfix: {}\nothers: {}'.format(len(same),len(prefix),len(infix),len(postfix),len(others)))
    return same, prefix, infix, postfix, others

def cmp_to_key(mycmp):
    'Convert a cmp= function into a key= function'
    class K:
        def __init__(self, obj, *args):
            self.obj = obj
        def __lt__(self, other):
            return mycmp(self.obj, other.obj) < 0
        def __gt__(self, other):
            return mycmp(self.obj, other.obj) > 0
        def __eq__(self, other):
            return mycmp(self.obj, other.obj) == 0
        def __le__(self, other):
            return mycmp(self.obj, other.obj) <= 0
        def __ge__(self, other):
            return mycmp(self.obj, other.obj) >= 0
        def __ne__(self, other):
            return mycmp(self.obj, other.obj) != 0
    return K

def mycmp(rhs,lhs):
    if rhs[1] == lhs[1]:
        for i in range(len(rhs[0])):
            if int(rhs[0][i]) > int(lhs[0][i]):
                return 1
            elif int(rhs[0][i]) == int(lhs[0][i]):
                continue
            else:
                return -1
        if len(rhs[0]) == len(lhs[0]):
            return 0
        else:
            return 1 if len(rhs[0]) > len(lhs[0]) else 0
    elif rhs[1] > lhs[1]:
        return 1
    else:
        return -1

def extract_remove(lst):
    remove, extract = [], []
    for i in range(len(lst)):
        integers = re.findall('([0-9]+)',lst[i])
        if len(integers) != 0:
            integer_lst = list(re.finditer('([0-9]+)',lst[i]))
            cpy = lst[i][:]
            for j in range(len(integer_lst)):
                cpy = cpy.replace(integer_lst[j].group(0),'',1)
            remove.append(cpy)
        else:
            remove.append(lst[i])
        extract.append(integers)
    return extract, remove
    
if __name__ == '__main__':
    case = 3
    print('case: {}'.format(case))
    map_in_name, script_name = '../cases/case{}.json'.format(case), 'test.py'
    nf = '../input/case{}_in.json'.format(case)
    nf = open(nf, 'r')
    nf = json.load(nf)
    keys, values = sorted(nf[0]), sorted(nf[1])
    
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()
    # rule based classifying
    same, prefix, infix, postfix, others = classify(origin_mapping)
   
    # only remove same
    same_set = set(same)
    exclusive_same = dict([item for item in origin_mapping.items() if item[0] not in same_set])
    
    # list of keys and values without exactly same rule
    kv_lst = sorted(list(exclusive_same.items()),key=lambda x: x[0])
    
    # divide kv_list to k and v
    k,v = zip(*kv_lst)
    k = list(k)
    v = list(v)   
    
    # ke: extraction of int of key
    # ve: extraction of int of value
    # kr: string removed int part of key
    # vr: string removed int part of value
    ke, kr = extract_remove(k)
    ve, vr = extract_remove(v)
    table = list(zip(ke,kr,ve,vr,k,v))
    table = sorted(table,key=lambda x: x[1])
    table = sorted(table,key=cmp_to_key(mycmp))      

    t0,t1,t2,t3,t4,t5 = zip(*table)
    check_table = list(zip(t2,t3,[i for i in range(len(k))]))
    check_table = sorted(check_table,key=lambda x: x[1])
    check_table = sorted(check_table,key=cmp_to_key(mycmp))
    ccc = 0
    for i in range(len(k)):
        if check_table[i][2] != i:
            print('*****[{}]*****'.format(i))   
            for j in range(i-10,i+10,1):
                print(check_table[j][0],check_table[j][1],check_table[j][2],j)
            print('*****[{}]*****'.format(i))
            ccc += 1
    print('shift: {}'.format(ccc))
    '''
    v, idx = zip(*mm) 
    
    idx = list(idx)
    v = list(v)
    mark = consecu_int_run_length(idx)
    print('len of str of mark: {}'.format(len(str(mark))))
    sporadic_seg = sporadic_seg_run_length(mark)
    print('len of str of sporadic_seg: {}'.format(len(str(sporadic_seg))))'''
    document1 = """import json as a,sys 
w={}
n,o=sys.argv[1:3]
k,v=a.load(open(n))
s=sorted
k,v=set(k),set(v)
r=k&v
c=list(r)
m=dict(zip(c,c))
k,v=s(k-r),s(v-r)
l=len
g=range
for i in g(l(k)):
    m[k[i]]=v[i]
for i in g(0,l(w),3):
    if w[i+2]!=1:
        for j in g(w[i+2]):
             m[k[w[i+1]+j]]=v[w[i]+j]
    else:
        for j in g(l(w[i+1])):
            m[k[w[i+1][j]]]=v[w[i]+j]
a.dump(m,open(o,'w'))""".format(str(sporadic_seg))

    document2 = """import json as a,sys
n,o=sys.argv[1:3]
k,v=a.load(open(n))
s=sorted
a.dump(dict(zip(s(k),s(v))),open(o,'w'))"""

    #script=open(script_name,'w')
    #script.write(document1 if len(sporadic_seg) != 0 else document2)
    #script.close()

case: 3
same: 3073
prefix: 7212
infix: 0
postfix: 0
others: 52441
*****[5532]*****
['2', '0', '1'] s_croy_fz/c_croy_bas/c_croy_grip/c_lp_bas/c_meo_dormice/c_meo_wamara/belk_mungy_nr_wr_qid__/T$ 5522 5522
['2', '1', '1'] s_croy_fz/c_croy_bas/c_croy_grip/c_lp_bas/c_meo_dormice/c_meo_wamara/belk_mungy_nr_wr_qid__/T$ 5523 5523
['2', '2', '1'] s_croy_fz/c_croy_bas/c_croy_grip/c_lp_bas/c_meo_dormice/c_meo_wamara/belk_mungy_nr_wr_qid__/T$ 5524 5524
['2', '3', '1'] s_croy_fz/c_croy_bas/c_croy_grip/c_lp_bas/c_meo_dormice/c_meo_wamara/belk_mungy_nr_wr_qid__/T$ 5525 5525
['3', '1'] s_croy_fz/c_croy_bas/c_croy_grip/c_lp_bas/c_meo_dormice/c_meo_wamara/bosque_kil_gust_wr_qid/T$ 5526 5526
['3', '1'] s_croy_fz/c_croy_bas/c_croy_grip/c_lp_bas/c_meo_dormice/c_meo_wamara/dari_jah_wr_qid/T$ 5527 5527
['3', '1'] s_croy_fz/c_croy_bas/c_croy_grip/c_lp_bas/c_meo_dormice/c_meo_wamara/dari_jr_ungt_wr_qid/T$ 5528 5528
['3', '1'] s_croy_fz/c_croy_bas/c_croy_grip/c_lp_bas/c_meo_dormice/c_meo_wamara/dari_raptly_wr_

['3', '3', '0', '1', '1'] s_tans_laid/tans_bas_laid/tans_bas/frey_tans_alea/frey_tans_caph_bas/frey_tans_caph_ns/zuza_caph_pc_tex_ait_ogres_qid__/T$ 40688 40688
['3', '3', '0', '2', '1'] s_tans_laid/tans_bas_laid/tans_bas/frey_tans_alea/frey_tans_caph_bas/frey_tans_caph_ns/zuza_caph_pc_tex_ait_ogres_qid__/T$ 40689 40689
['3', '3', '0', '3', '1'] s_tans_laid/tans_bas_laid/tans_bas/frey_tans_alea/frey_tans_caph_bas/frey_tans_caph_ns/zuza_caph_pc_tex_ait_ogres_qid__/T$ 40690 40690
['3', '3', '0', '4', '1'] s_tans_laid/tans_bas_laid/tans_bas/frey_tans_alea/frey_tans_caph_bas/frey_tans_caph_ns/zuza_caph_pc_tex_ait_ogres_qid__/T$ 40691 40691
['3', '3', '1', '0', '1'] s_tans_laid/tans_bas_laid/tans_bas/frey_tans_alea/frey_tans_caph_bas/frey_tans_caph_ns/zuza_caph_pc_tex_ait_ogres_qid__/T$ 40692 40692
['3', '3', '1', '1', '1'] s_tans_laid/tans_bas_laid/tans_bas/frey_tans_alea/frey_tans_caph_bas/frey_tans_caph_ns/zuza_caph_pc_tex_ait_ogres_qid__/T$ 40693 40693
['3', '3', '1', '2', '1'] s_tans_l